In [ ]:
!pip install torch

In [ ]:
import torch

In [ ]:
mu_shape = (2924,1,2,20)
sigma_shape = (2924,1,2,20)
pi_shape = (2924,20)

In [ ]:
pi = torch.rand(pi_shape)
mu = torch.randn(mu_shape)
sigma = torch.randn(sigma_shape)

In [ ]:
pi_indexes = pi.max(dim = 1, keepdim = True)[1]

In [ ]:
component_mu = mu.index_select(dim = 3, index = pi_indexes[-1]).squeeze(dim = 3)

In [ ]:
component_sigma = sigma.index_select(dim = 3, index = pi_indexes[-1]).squeeze(dim = 3)

In [ ]:
sigma.size(), mu.size()

In [ ]:
def adjust_temp(pi_pdf, temp):
    # Update to log functinos were made to support 0 values
    pi_pdf = torch.log1p(pi_pdf)/temp
    pi_pdf -= torch.max(pi_pdf, dim = -1, keepdim=True)[0]
    pi_pdf = torch.exp(pi_pdf) - 1.0
    pi_pdf /= torch.sum(pi_pdf, dim = -1, keepdim = True)
    return pi_pdf

In [ ]:
probs = adjust_temp(pi, 0.5)
logits = torch.log1p(probs)

In [ ]:
comp_indexes = torch.multinomial(logits, 1).reshape(-1, 1)

In [ ]:
component_mu.size()

In [ ]:
torch.normal(mean = component_mu, std = component_sigma*0.5*0.5)

In [ ]:
comp_indexes.size()

In [ ]:
pi_pdf.size()

In [ ]:
pi_pdf.size()

In [ ]:
torch.sum(pi_pdf, dim = -1, keepdim = True).size()

In [ ]:
pi_pdf /= torch.sum(pi_pdf, dim = -1, keepdim = True)

In [ ]:
pi_pdf = torch.log(pi)/0.5

In [ ]:
pi_pdf

In [ ]:
torch.max(pi_pdf, dim = -1, keepdim=True)[0]

In [ ]:
pi_pdf_v2 = pi_pdf.sub(torch.max(pi_pdf, dim = -1, keepdim=True)[0]) 

In [ ]:
pi_pdf

In [ ]:
adjust_temp(pi, 0.5)

In [ ]:
pi

In [ ]:
pi_pdf = torch.log(pi + 1.0)/.5

In [ ]:
torch.max(pi_pdf, dim = -1)[0].shape

In [ ]:
m = torch.tensor([[0, 0, 0, 0],[1, 0,1, 0]]).to(device='cpu')

In [ ]:
pi_pdf = torch.log(m)

In [ ]:
pi_pdf = torch.log(pi_pdf)/temp

In [ ]:
pi_pdf

In [ ]:
torch.max(pi_pdf, dim = -1)[0]

In [ ]:
torch.max(pi_pdf)

In [ ]:
def draw_sample(outputs: dict, greedy:bool=False, greedy_mu:bool=True, temp:float =0.5):
    '''
    Obtains 2D strokes from mu, sigma, pis returned by GMM
    '''
    is_2d = True
    if outputs["mu"].dim() == 3:
        is_2d = False
    out_shape = outputs["mu"].size()
    batch_size = out_shape[0]
    seq_len = 1 if is_2d else out_shape[1]
    comp_shape = (batch_size, seq_len, 2, 20)
    #reshapes mu and sigma according to comp_shape
    pi = outputs["pi"]
    mu = outputs["mu"].reshape(comp_shape)
    sigma = outputs["sigma"].reshape(comp_shape)
    #permute variables (?, seq_len, out_units, num_components)
    mu = mu.permute(0, 1, 3, 2) 
    sigma = sigma.permute(0, 1, 3, 2)
    probs = pi.reshape(-1, 20)
    #when greedy: select mus, and sigmas according to max probabilites
    if greedy:
        logits = torch.log1p(probs) 
        comp_indexes = logits.max(dim = 1, keepdim = True)[1]
    #when not greedy: selects mus, and sigmas according to a categorial distribution with probabilities
    else:
        probs_adjusted = adjust_temp(pi_pdf, temp)
        logits = torch.log1p(probs_adjusted)
        comp_indexes = torch.multinominal(logits, 1).reshape(-1, seq_len) #multinomial distribution is categorical
    #selects components of mu and sigma according to indexes selected
    component_mu = mu.index_select(dim = 3, index = comp_indexes[-1]).squeeze(dim = 3)
    component_sigma = sigma.index_select(dim = 3, index = comp_indexes[-1]).squeeze(dim = 3)
    #when greedy_mu component mu is the one calculated
    if greedy_mu:
        sample = component_mu
    #when not greedy_mu component mu is equal to a normal distribution with mean=component_mu and std=component_sigma*temp^2
    else:
        sample = torch.normal(mean = component_mu, std = component_sigma*(temp^2))
    if is_2d:
        return sample.squeeze() #output shape: (?, 2)
    else:
        return sample

In [ ]:
import torch.nn as nn

In [ ]:
layer_features = [512,512,512,512]

In [ ]:
[nn.Linear(
    in_features=20, 
    out_features=layer_features[0])]

In [ ]:
nn.ModuleList([nn.Linear(
    in_features=20, 
    out_features=layer_features[0])] +\
[nn.Linear(
    in_features=layer_features[i], 
    out_features= layer_features[i+1]) for i in range(len(layer_features)- 1)])

In [ ]:
nn.ModuleList(
    [nn.Linear(
        in_features=size_embedding, 
        out_features=layer_features[0])] +
    [nn.Linear(in_features=layer_features[i+1], out_features= layer_features[i+2]) for i in layer_features]
)

In [ ]:
import numpy as np

# Preparing for Relational data

#### Creating dummy data

In [ ]:
import torch

In [ ]:
num_diagrams = 43
batch_enc_out = 731
embedding_size = 8
min_strokes = 12
max_strokes = 17

In [ ]:
encoder_out = torch.randn((batch_enc_out, embedding_size)) #encoder output tensor
inputs_start_coord = torch.randn(batch_enc_out,1,2) #input start_coord tensor
inputs_end_coord =  torch.randn(inputs_start_coord.shape) #input end_coord tensor
num_strokes_x_diagram_tensor = torch.randint(min_strokes,max_strokes + 1, size = (num_diagrams,)) #input input_n_strokes

### flujo completo reducido

In [ ]:
config = {"input_type": "hybrid",
                 "num_predictive_inputs": 32,
                 "replace_padding": True,
                 "end_positions": False,
                 "stop_predictive_grad": False}

In [ ]:
pred_inputs, pred_input_seq_len, context_pos, pred_targets = random_index_sampling(encoder_out,inputs_start_coord,
                                                                                   inputs_end_coord,num_strokes_x_diagram_tensor,
                                                                                   input_type =config["input_type"],
                                                                                   num_predictive_inputs = config["num_predictive_inputs"],
                                                                                   replace_padding = config["replace_padding"],
                                                                                   end_positions = config["end_positions"]
                                                                                  )
pred_targets.requires_grad = False
if loaded_config["stop_predictive_grad"]:
    pred_inputs.requires_grad = False

### To copy

In [ ]:
num_diagrams = 43
batch_enc_out = 731
embedding_size = 8
min_strokes = 12
max_strokes = 17
#
encoder_out = torch.randn((batch_enc_out, embedding_size)) #encoder output tensor
inputs_start_coord = torch.randn(batch_enc_out,1,2) #input start_coord tensor
inputs_end_coord =  torch.randn(inputs_start_coord.shape) #input end_coord tensor
num_strokes_x_diagram_tensor = torch.randint(min_strokes,max_strokes + 1, size = (num_diagrams,)) #input input_n_strokes
#
config = {"input_type": "hybrid",
                 "num_predictive_inputs": 32,
                 "replace_padding": True,
                 "end_positions": False,
                 "stop_predictive_grad": False}
#
pred_inputs, pred_input_seq_len, context_pos, pred_targets = random_index_sampling(encoder_out,inputs_start_coord,
                                                                                   inputs_end_coord,num_strokes_x_diagram_tensor,
                                                                                   input_type =config["input_type"],
                                                                                   num_predictive_inputs = config["num_predictive_inputs"],
                                                                                   replace_padding = config["replace_padding"],
                                                                                   end_positions = config["end_positions"]
                                                                                  )
pred_targets.requires_grad = False
if config["stop_predictive_grad"]:
    pred_inputs.requires_grad = False

# Transformer

# Preparing data for embedding

In [3]:
import json
config_ = json.load(open("config.json", 'rb'))

In [4]:
import pickle
import os
from models import *

In [5]:
cose_model  = CoSEModel(**config_)

In [6]:
inputs = pickle.load(open(os.path.join("D:\Projects\RC-PWC2020\data", "inputs_list_based.pkl"), 'rb'))
targets = pickle.load(open(os.path.join("D:\Projects\RC-PWC2020\data", "target_list_based.pkl"), 'rb'))

In [7]:
sample_input = inputs[0]
sample_target = targets[0]

In [8]:
import torch

In [11]:
input_, target_ = batch_diagram_to_stroke(sample_input, sample_target, 3)

In [12]:
enc_inputs = torch.from_numpy(input_['encoder_inputs'])
t_inputs = torch.from_numpy(input_['t_input'])
stroke_len_inputs = torch.from_numpy(input_['seq_len'])

In [13]:
def generate_square_subsequent_mask(sz):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [14]:
def generate_3d_mask(seq_input, seq_len_input):
    look_ahead_mask = generate_square_subsequent_mask(seq_input.shape[1])
    seq_mask = 1 - (torch.arange(stroke_len_inputs.max().item())[None, :] < stroke_len_inputs[:, None]).float()
    seq_mask  = seq_mask.masked_fill(seq_mask == 1, float('-inf')).unsqueeze(dim=2).repeat(1,1,seq_mask.shape[1])
    return torch.minimum(seq_mask, look_ahead_mask), look_ahead_mask, seq_mask

In [93]:
src_3d_mask, look_ahead_mask, seq_mask = generate_3d_mask(enc_inputs, stroke_len_inputs)

In [129]:
out_encoder = cose_model.encoder(enc_inputs.permute(1,0,2), stroke_len_inputs, look_ahead_mask)

torch.Size([97, 768, 64])
tensor([[[ 0.0530,  1.5284, -1.4227,  ..., -0.9789, -1.0885, -0.8036],
         [ 0.1772,  1.5447, -1.3344,  ..., -0.8030, -1.0910, -0.7979],
         [ 0.3444,  1.3610, -1.4185,  ..., -1.0445, -0.8237, -1.3872],
         ...,
         [-0.1718,  0.4918, -1.3834,  ..., -0.7728,  1.6515,  0.4093],
         [-0.0378,  0.4513, -1.4079,  ..., -0.7473,  1.6977,  0.2563],
         [-0.1174,  0.1912, -1.6401,  ..., -0.7678,  1.7473,  0.1135]],

        [[ 0.0530,  1.5284, -1.4227,  ..., -0.9789, -1.0885, -0.8036],
         [ 0.2134,  0.8767, -1.7480,  ..., -1.3055, -0.2065, -1.6600],
         [ 0.2260,  0.1671, -2.0990,  ..., -1.2285,  0.4287, -1.7383],
         ...,
         [-0.3428, -1.6862, -1.7424,  ..., -1.2098,  2.3888,  1.1468],
         [-0.2578, -1.6595, -1.8023,  ..., -1.1795,  2.4979,  1.0440],
         [-0.2620, -1.8088, -2.0301,  ..., -1.1767,  2.4686,  0.8790]],

        [[ 0.0530,  1.5284, -1.4227,  ..., -0.9789, -1.0885, -0.8036],
         [ 0.2634, 

In [131]:
out_encoder

tensor([[ 0.5962,  1.6801,  0.5153,  ..., -1.4957, -0.4100,  0.0826],
        [ 0.2414,  0.7255,  0.1064,  ..., -0.6304,  0.0044,  0.5023],
        [ 0.6303,  0.8308,  0.5827,  ..., -0.8371, -0.2658, -0.2817],
        ...,
        [ 0.6275,  1.5211,  0.6182,  ..., -1.3546, -0.2110,  0.0343],
        [ 0.4640,  1.3644,  0.5649,  ..., -1.3547, -0.2803,  0.0281],
        [ 0.4412,  1.4317,  0.7230,  ..., -1.4038, -0.6683,  0.3492]],
       grad_fn=<AddmmBackward>)

In [113]:
pos_pred = torch.randn(1,2)

In [120]:
gmm = torch.randn(1,9,258)

In [122]:
torch.cat([gmm, pos_pred.unsqueeze(dim = 1).repeat(1,9,1)], dim = 2)

torch.Size([1, 9, 260])

In [53]:
num_components = 10
out_size = 2

In [82]:
outputs ={"mu": torch.randn(1, num_components*out_size), "sigma": torch.randn(1,num_components*out_size),
             "pi": torch.randn(1,num_components)}

In [83]:
draw_sample(outputs, greedy = True).shape

torch.Size([1, 1, 10, 2])
torch.Size([1, 1])
torch.Size([1, 1, 2])


torch.Size([1, 2])

In [80]:
def draw_sample(outputs: dict, greedy:bool=False, greedy_mu:bool=True, temp:float =0.5):
    '''
    Obtains 2D strokes from mu, sigma, pis returned by GMM
    '''
    is_2d = True
    if outputs["mu"].dim() == 3:
        is_2d = False
    out_shape = outputs["mu"].size()
    batch_size = out_shape[0]
    seq_len = 1 if is_2d else out_shape[1]
    comp_shape = (batch_size, seq_len, out_size, num_components)
    #reshapes mu and sigma according to comp_shape
    pi = outputs["pi"]
    mu = outputs["mu"].reshape(comp_shape)
    sigma = outputs["sigma"].reshape(comp_shape)
    #permute variables (?, seq_len, out_units, num_components)
    mu = mu.permute(0, 1, 3, 2)
    print(mu.shape)
    sigma = sigma.permute(0, 1, 3, 2)
    probs = pi.reshape(-1, num_components)
    #when greedy: select mus, and sigmas according to max probabilites
    if greedy:
        logits = torch.log1p(probs) 
        comp_indexes = logits.max(dim = 1, keepdim = True)[1]
    #when not greedy: selects mus, and sigmas according to a categorial distribution with probabilities
    else:
        probs_adjusted = adjust_temp(pi_pdf, temp)
        logits = torch.log1p(probs_adjusted)
        comp_indexes = torch.multinominal(logits, 1).reshape(-1, seq_len) #multinomial distribution is categorical
    #selects components of mu and sigma according to indexes selected
    print(comp_indexes.shape)
    component_mu = mu.index_select(dim = 2, index = comp_indexes[-1]).squeeze(dim = 2)
    print(component_mu.shape)
    component_sigma = sigma.index_select(dim = 2, index = comp_indexes[-1]).squeeze(dim = 2)
    #when greedy_mu component mu is the one calculated
    if greedy_mu:
        sample = component_mu
    #when not greedy_mu component mu is equal to a normal distribution with mean=component_mu and std=component_sigma*temp^2
    else:
        sample = torch.normal(mean = component_mu, std = component_sigma*(temp^2))
    if is_2d:
        return sample.squeeze(dim = 1) #output shape: (?, 2)
    else:
        return sample

In [22]:
seq_mask.shape

torch.Size([74496, 97, 97])

In [1]:
seq_mask.shape, 

NameError: name 'seq_mask' is not defined

In [97]:
torch.minimum(seq_mask, look_ahead_mask)

RuntimeError: The size of tensor a (768) must match the size of tensor b (97) at non-singleton dimension 1

In [72]:
seq_mask = 1 - (torch.arange(stroke_len_inputs.max().item())[None, :] < stroke_len_inputs[:, None]).float()

In [47]:
out_encoder = cose_model.encoder(enc_inputs.permute(1,0,2), stroke_len_inputs, look_ahead_mask)

torch.Size([97, 768, 64])
tensor([[[ 0.1954, -0.4692,  0.0050,  ...,  0.6069, -0.7033,  0.0290],
         [ 0.3912, -0.4092,  0.2105,  ...,  0.6060, -0.8481, -0.0680],
         [ 0.3291, -1.1856,  0.0725,  ...,  0.2461, -0.2257,  0.4046],
         ...,
         [-0.0497, -0.2687,  0.4462,  ...,  0.1759, -0.4275, -0.0027],
         [ 0.1757, -0.2384,  0.5353,  ...,  0.1459, -0.4374, -0.0524],
         [ 0.2562, -0.4149,  0.4614,  ...,  0.0879, -0.3387, -0.0135]],

        [[ 0.1954, -0.4692,  0.0050,  ...,  0.6069, -0.7033,  0.0290],
         [-0.1080, -0.7571,  0.7001,  ..., -0.4141, -0.4302,  0.0192],
         [-0.1558, -0.6740,  0.9561,  ..., -0.5954, -0.5528,  0.2184],
         ...,
         [ 0.1108,  0.2183,  1.2628,  ..., -0.4207, -1.1018,  0.5429],
         [ 0.2621,  0.2270,  1.2946,  ..., -0.4309, -1.1176,  0.4959],
         [ 0.3240,  0.1114,  1.2112,  ..., -0.4660, -1.0362,  0.5310]],

        [[ 0.1954, -0.4692,  0.0050,  ...,  0.6069, -0.7033,  0.0290],
         [ 0.3439, 

In [48]:
out_encoder

tensor([[ 1.1809, -0.1351,  0.2037,  ..., -0.7732, -0.4463,  0.2469],
        [ 0.6715, -0.1332,  0.5573,  ..., -0.8245, -0.3798,  0.1617],
        [ 0.7992,  0.0052,  0.7707,  ..., -0.6442, -1.2051, -0.1041],
        ...,
        [ 1.0841, -0.0080,  0.1744,  ..., -0.7723, -0.6316,  0.2348],
        [ 1.3098, -0.0625,  0.2096,  ..., -0.8533, -0.6813,  0.1241],
        [ 1.0203, -0.2480,  0.2407,  ..., -0.7391, -0.5876,  0.5197]],
       grad_fn=<AddmmBackward>)

In [ ]:
torch.sum(out_encoder)

In [10]:
  def batch_diagram_to_stroke(inputs, targets, model_inp_size):
    """Converts batch of diagrams into batch of strokes.
    Reshapes [num_diagrams, num_strokes, ...] shaped inputs and targets into
    [num_diagrams x num_strokes, ...]. Note that num_strokes are padded to the
    maximum number of strokes in the batch.
    Args:
      inputs (dict): model inputs, i.e., return of __to_model_batch.
      targets (dict): model targets, i.e., return of __to_model_batch.
    Returns:
      (tuple) of reshaped inputs and targets.
    """
    n_samples = inputs['encoder_inputs'].shape[0]
    n_strokes = inputs['encoder_inputs'].shape[1]
    batch_dim =  n_samples*n_strokes
    
    inputs['encoder_inputs'] = inputs['encoder_inputs'].reshape([batch_dim, -1, model_inp_size])
    inputs['decoder_inputs'] = inputs['decoder_inputs'].reshape([batch_dim, -1, model_inp_size])
    inputs['seq_len'] = inputs['seq_len'].reshape([batch_dim])
    inputs['start_coord'] = inputs['start_coord'].reshape([batch_dim, 1, 2])
    inputs['end_coord'] = inputs['end_coord'].reshape([batch_dim, 1, 2])
    
    targets["stroke"] = targets["stroke"].reshape([batch_dim, -1, 2])
    targets["pen"] = targets["pen"].reshape([batch_dim, -1, 1])
    targets['seq_len'] = targets['seq_len'].reshape([batch_dim])
    targets['start_coord'] = targets['start_coord'].reshape([batch_dim, 1, 2])
    targets['end_coord'] = targets['end_coord'].reshape([batch_dim, 1, 2])

    inputs['t_input'] = inputs['t_input'].reshape([batch_dim, -1])
    inputs['t_target_ink'] = inputs['t_target_ink'].reshape([-1, 3])
    targets['t_target_ink'] = targets['t_target_ink'].reshape([-1, 3])
    targets['t_target_pen'] = targets['t_target_pen'].reshape([-1, 1])
    targets['t_target_stroke'] = targets['t_target_stroke'].reshape([-1, 2])
    targets["stroke_mask"] = torch.where(torch.from_numpy(inputs['t_input']) > 0., torch.ones_like(torch.from_numpy(inputs['t_input'])),torch.zeros_like(torch.from_numpy(inputs['t_input']))).reshape([-1])

    return inputs, targets

### processing

In [ ]:
#config variables 
input_type = "hybrid"
num_predictive_inputs = 32 #DEPENDS HEAVILY ON DATA
replace_padding = True
end_positions = False

In [ ]:
diagram_embedding, padded_max_num_strokes, min_n_stroke, num_diagrams = reshape_stroke2diagram(encoder_out,num_strokes_x_diagram_tensor)

In [ ]:
all_n_inputs = []
all_input_indexes = []
all_target_indexes = []
all_seq_len = []

if input_type == "hybrid":
    num_predictive_inputs //= 2

if input_type in ["random", "hybrid"]:
    for i in range(num_predictive_inputs):
        input_indexes, target_indexes, n_inputs = get_random_inp_target_pairs(num_strokes_x_diagram_tensor,
                                                                              padded_max_num_strokes,
                                                                              num_diagrams,
                                                                              min_n_stroke)
        all_input_indexes.append(input_indexes)        
        all_target_indexes.append(target_indexes)
        all_n_inputs.append(n_inputs)
        all_seq_len.append(torch.ones([num_diagrams])*n_inputs)
    
if input_type in ["order", "hybrid"]:
    for i in range(num_predictive_inputs):
        input_indexes, target_indexes, n_inputs = get_ordered_inp_target_pairs(num_strokes_x_diagram_tensor,
                                                                               padded_max_num_strokes,
                                                                               num_diagrams,
                                                                               min_n_stroke)
        all_input_indexes.append(input_indexes)
        all_target_indexes.append(target_indexes)
        all_n_inputs.append(n_inputs)
        all_seq_len.append(torch.ones([num_diagrams])*n_inputs)

In [ ]:
#preparing for tensor indexing
input_range_n_inputs = torch.arange(start=0, end = num_diagrams).repeat(1,len(all_n_inputs)).permute(1,0).squeeze()
gather_target_index = torch.stack([input_range_n_inputs,
             torch.cat(all_target_indexes, dim = 0).squeeze()], dim = -1)
start_pos_base = inputs_start_coord.reshape(num_diagrams,padded_max_num_strokes,2)
end_pos_base = inputs_end_corrd.reshape(num_diagrams,padded_max_num_strokes,2)

In [ ]:
#gathering indexes from base tensors
pred_targets = torch.stack([diagram_embedding[i][j] for i,j in gather_target_index])
pred_inputs = gather_indexes(diagram_embedding, all_input_indexes, replace_padding = True)
pred_input_seq_len = torch.cat(all_seq_len,dim=0)
if end_positions:
    start_pos = gather_indexes(start_pos_base, all_input_indexes, replace_padding = True)
    end_pos = gather_indexes(end_pos_base, all_input_indexes, replace_padding = True)
    context_pos = torch.cat([start_pos, end_pos], dim=-1)
else:
    context_pos = gather_indexes(start_pos_base, all_input_indexes, replace_padding = True)

In [ ]:
def random_index_sampling(encoder_out,inputs_start_coord,inputs_end_coord,num_strokes_x_diagram_tensor,
                         input_type ="hybrid", num_predictive_inputs = 32, replace_padding = True, end_positions = False):
    
    #obtains diagram embedding (batch_strokes, embedding_size) -> (num_diagrams, padded_n_strokes, embedding_size)
    diagram_embedding, padded_max_num_strokes, min_n_stroke, num_diagrams = reshape_stroke2diagram(encoder_out,num_strokes_x_diagram_tensor)
    #creates indexes to gather from
    all_n_inputs = []
    all_input_indexes = []
    all_target_indexes = []
    all_seq_len = []

    if input_type == "hybrid":
        num_predictive_inputs //= 2

    if input_type in ["random", "hybrid"]:
        for i in range(num_predictive_inputs):
            input_indexes, target_indexes, n_inputs = get_random_inp_target_pairs(num_strokes_x_diagram_tensor,
                                                                                  padded_max_num_strokes,
                                                                                  num_diagrams,
                                                                                  min_n_stroke)
            all_input_indexes.append(input_indexes)        
            all_target_indexes.append(target_indexes)
            all_n_inputs.append(n_inputs)
            all_seq_len.append(torch.ones([num_diagrams])*n_inputs)

    if input_type in ["order", "hybrid"]:
        for i in range(num_predictive_inputs):
            input_indexes, target_indexes, n_inputs = get_ordered_inp_target_pairs(num_strokes_x_diagram_tensor,
                                                                                   padded_max_num_strokes,
                                                                                   num_diagrams,
                                                                                   min_n_stroke)
            all_input_indexes.append(input_indexes)
            all_target_indexes.append(target_indexes)
            all_n_inputs.append(n_inputs)
            all_seq_len.append(torch.ones([num_diagrams])*n_inputs)

    #preparing for tensor indexing
    input_range_n_inputs = torch.arange(start=0, end = num_diagrams).repeat(1,len(all_n_inputs)).permute(1,0).squeeze()
    gather_target_index = torch.stack([input_range_n_inputs,
                 torch.cat(all_target_indexes, dim = 0).squeeze()], dim = -1)
    start_pos_base = inputs_start_coord.reshape(num_diagrams,padded_max_num_strokes,2)
    end_pos_base = inputs_end_coord.reshape(num_diagrams,padded_max_num_strokes,2)
    #gathering indexes from base tensors
    pred_targets = torch.stack([diagram_embedding[i][j] for i,j in gather_target_index])
    pred_inputs = gather_indexes(diagram_embedding, all_input_indexes, replace_padding = True)
    pred_input_seq_len = torch.cat(all_seq_len,dim=0)
    if end_positions:
        start_pos = gather_indexes(start_pos_base, all_input_indexes, replace_padding = True)
        end_pos = gather_indexes(end_pos_base, all_input_indexes, replace_padding = True)
        context_pos = torch.cat([start_pos, end_pos], dim=-1)
    else:
        context_pos = gather_indexes(start_pos_base, all_input_indexes, replace_padding = True)
    return pred_inputs, pred_input_seq_len, context_pos, pred_targets

def gather_indexes(base_tensor, index_tensor, replace_padding = True):
    '''
    Simulates gather_nd and has an extra property to replace padding with values in first (stroke) in first (diagram)
    Args:
        index_tensor: list of var-len tensors
        base_tensor: tensor from which to sample from
    Returns:
        gathered_tensor_padded: tensor sampled and replaced in padding if option replace_padding = True
    '''
    tensor_var_len = [base_tensor[i,value,:].squeeze() for a in index_tensor for i,value in enumerate(a)]
    gathered_tensor_padded = torch.nn.utils.rnn.pad_sequence(tensor_var_len, batch_first=False, padding_value=0).permute(1,0,2)
    if replace_padding:
        gathered_tensor_padded = replacing_padding_with_embedding(base_tensor, gathered_tensor_padded, index_diagram = 0, index_stroke = 0)    
    return gathered_tensor_padded

def replacing_padding_with_embedding(ref_tensor, tensor_to_mod, index_diagram= 0, index_stroke= 0):
    '''
    replaces padded values with values from dia
    '''
    default_first_base_tensor = ref_tensor[0,0,:].repeat(tensor_to_mod.shape[0],tensor_to_mod.shape[1],1)
    return torch.where(tensor_to_mod != 0.0, tensor_to_mod, default_first_base_tensor)

def reshape_stroke2diagram(stroke_embedding,num_strokes_x_diagram_tensor):
    padded_max_n_strokes = torch.max(num_strokes_x_diagram_tensor).item()
    min_n_stroke = torch.min(num_strokes_x_diagram_tensor).item()
    num_diagrams = num_strokes_x_diagram_tensor.shape[0]
    diagram_embedding = stroke_embedding.reshape([num_diagrams, padded_max_n_strokes, embedding_size])
    return diagram_embedding, padded_max_n_strokes, min_n_stroke, num_diagrams

def get_random_inp_target_pairs(num_strokes_x_diagram_tensor, padded_max_num_strokes, num_diagrams, min_n_stroke):
    """Get a randomly generated input set and a target."""
    n_inputs = torch.randint(2, (min_n_stroke+1),size = (1,)).item()
    target_indexes = (torch.rand([num_diagrams])*num_strokes_x_diagram_tensor).int().reshape(-1,1)
    input_range = torch.arange(start=1, end = padded_max_num_strokes + 1).repeat(num_diagrams,1)
    mask = (input_range <= num_strokes_x_diagram_tensor.reshape(-1,1)) & (input_range != target_indexes)
    input_indexes = torch.multinomial((input_range*mask).float(),n_inputs) - 1
    return input_indexes, target_indexes, n_inputs

def get_ordered_inp_target_pairs(num_strokes_x_diagram_tensor, padded_max_num_strokes, num_diagrams, min_n_stroke, random_target = False):
    """Get a slice (i.e., window) randomly."""
    n_inputs = torch.randint(2, (min_n_stroke+1),size = (1,)).item()
    start_index = torch.randint(0, min_n_stroke - n_inputs + 1, size = (1,)).item()
    if not random_target:
        target_indexes = torch.tensor([start_index+n_inputs]).repeat(num_diagrams,1)
    else:
        target_indexes = (torch.rand([num_diagrams])*num_strokes_x_diagram_tensor).int().reshape(-1,1)
    input_range = torch.arange(start=1, end = padded_max_num_strokes + 1).repeat(num_diagrams,1)
    mask = ((input_range - 1)< target_indexes) & ((input_range - 1)>= start_index)
    input_indexes = input_range.masked_select(mask).reshape(num_diagrams, n_inputs)
    return input_indexes, target_indexes, n_inputs